In [ ]:
import openai
import gradio as gr
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
import re
os.environ['OPENAI_API_KEY'] = ''

### 11/30 分享與實作
1. 主機套裝推薦優化
2. 組機套裝諮詢 - 不同風格的智能體推薦

### 主機套裝推薦優化

In [2]:
component_dict = {
    "CPU": {"name": "Intel Core i5-13500 [14核/20執行緒] 2.5GHz 4.8GHz/24M/UHD770 [盒裝盒)", 
            "price": 7700},
    "Mother board": {"name": "技嘉 B760M GAMING X DDR4-ATX/Realtek 2.5GbE+千兆+Wi-Fi 6E(含背板)",
            "price":4990
            },
    "Fan": {"name": "DEEPCOOL 九州風 AG620 CPU散熱器(含6*12公分風扇/2相/銅底/高塔型/157mm)",
            "price": 1390},
    "Memory": {"name": "十铨 T-Force Vulcan Z 火神系列 DDR4-32GB(16GB*2) CL16",
               "price": 1999},
    "Power": {"name": "全漢 HYDRO GSM PRO 650W/金牌",
              "price": 1990},
    "GPU": {"name": "技嘉 RTX 4070 WINDFORCE OC 12G(2490MHz/26.1cm/8pin/三風扇/四年保)",
            "price": 19990},
    "Device": {"name": "威剛 XPG Core Reactor 5 Plus 1TB PCIe 4.0 x4/6600M/5000M/3D TLC/五年保固",
               "price": 1888},
    "Case": {"name": "酷碼 SW300 M/白 玻璃側透/M-ATX/",
             "price": 1500},
}

In [3]:
prompt = """
以下的元件為{component_name}，
請針對這個產品型號，與價格，推薦三個價格差不多相近的產品：
產品型號：{name}
產品價格：{price}

請提供產品的資訊，包跨產品型號與產品價格
範例格式：

推薦清單：
1. 產品型號：xxx | 產品價格：xxx
2. 產品型號：xxx | 產品價格：xxx
3. 產品型號：xxx | 產品價格：xxx

"""

template = ChatPromptTemplate.from_template(prompt)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
parser = StrOutputParser()

chain = template | model | parser

In [4]:
component_list = list(component_dict.keys())
recommend_str = ''

for component_name in component_list:
    
    name = component_dict[component_name]['name']
    price = component_dict[component_name]['price']

    message = chain.invoke({"component_name":component_name, "name":name, "price":price})

    # 正規表達法，取出產品資訊
    pattern = r"產品型號：([\w\s\-]+) \| 產品價格：(\d+)"
    matches = re.findall(pattern, message)

    list_data = [{"name": match[0], "price": int(match[1])} for match in matches]

    # 把結構化資訊返回成字串
    component_recommend = f'{component_name} 推薦清單：' 
    for i, data in enumerate(list_data):
        component_recommend += f'\n產品名稱： {data["name"]} | 產品價格： {data["price"]}'
    
    recommend_str += f'{component_recommend}\n'

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************g64A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
present_list = '目前的電腦組合清單：\n'
for key, value in component_dict.items():
    present_list += f'{key} | 產品名稱：{value["name"]} | 產品價格：{value["price"]}\n'

In [ ]:
summary_prompt = """
現在，你擁有一組目前的電腦組合清單，現在有的零件為：{component_list}
以下為現有的電腦組合清單：
{present_list}

每一個元件的推薦清單：
{recommend_str}

請針對現在有的電腦組合清單，從以上元件的推薦清單中，塞選並且挑出 4~8 個你覺得可以替換的零件，並說明原因，
原因的文字敘述請限定在 60字以內。

返回的格式由 元件名稱 | 產品名稱 | 產品價格所組成
以下為返回的範例：
1. CPU | AMD 5600 | 13500 | 原因：AMD 5600 CP值高
2. GPU | Nvidia 4090 | 58500 | 原因： GPU Nvidia 40900 現在性能很好
...

且記，不用所有的元件都提供一個推薦清單，只要是值得被替換的推薦零件，
同一個元件有多個推薦零件是沒有問題的。

"""
template = ChatPromptTemplate.from_template(summary_prompt)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
parser = StrOutputParser()

summary_agent = template | model | parser

recommend_list = summary_agent.invoke({"component_list":component_list, "present_list":present_list, "recommend_str":recommend_str})
print(recommend_list)

1. CPU | Intel Core i5-13400 | 7800 | 原因：性能接近i5-13500，價格略高，性價比好。
2. Mother board | ASRock B760M Pro4 WiFi | 4790 | 原因：價格更低，性能足夠，性價比高。
3. Fan | Thermalright Assassin 120 SE | 1350 | 原因：散熱性能優異，價格合理。
4. Power | 安鈦克 ANTEC EA-650G | 2050 | 原因：品質可靠，650W功率足夠，價格合理。
5. GPU | 華碩 RTX 4070 DUAL OC 12G | 19800 | 原因：稍便宜，但性能相似，性價比高。


### 組機套裝諮詢 - 不同風格的智能體推薦

In [ ]:
prompt = """
你是一個 {setting}，
使用者想請教，關於電腦零件採買的問題，
請根據使用者的回覆，給予一個你推薦的零件，並說明原因。

使用者詢問：{user_question}
請把字數限定在100字以內。
"""

template = ChatPromptTemplate.from_template(prompt)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
parser = StrOutputParser()

agent = template | model | parser

# user_question = '我想要買一萬五千左右的CPU'
# setting = '在採買電腦零件，優先考慮外型酷炫配件的的人'

# recommend = agent.invoke({"setting":setting, 'user_question': user_question})
# print(recommend)

我推薦你購買AMD Ryzen 5 5600X。這款CPU具備6核心12執行緒，性能強勁，適合遊戲和多任務處理。它的外型設計也相當酷炫，搭配RGB散熱器更能增添電腦的美感。此外，價格在一萬五千左右，性價比非常高，讓你可以在不妥協性能的情況下，打造一台帥氣的電腦。


In [ ]:

agent_1_setting = '在採買電腦零件時，優先考慮外型酷炫配件的的人'
agent_2_setting = '在採買電腦零件，優先考慮性價比的人'
agent_3_setting = '在採買電腦零件，優先考慮性能的人'
0
mulit_agent_setting = [agent_1_setting, agent_2_setting, agent_3_setting]

In [ ]:
user_question = '我想要買三萬左右的CPU'

for i, agent_setting in enumerate(mulit_agent_setting):

    recommend = agent.invoke({"setting":agent_setting, 'user_question': user_question})
    print(f'Agent {i+1}: {recommend}')

Agent 1: 我推薦你選擇AMD Ryzen 5 7600X。這款CPU不僅性能強勁，適合遊戲和多任務處理，還擁有酷炫的外觀設計，特別是搭配RGB散熱器時，能讓整個電腦更具視覺衝擊力。而且價格大約在三萬左右，性價比非常高，絕對能滿足你對外型和性能的雙重需求！
Agent 2: 我推薦AMD Ryzen 5 5600X。這款CPU在性能與價格之間提供了極佳的平衡，適合遊戲和多任務處理。它擁有6核心12執行緒，基礎時脈和加速時脈表現優異，並且相對於同級別的Intel處理器，價格更具優勢，性價比高，非常適合三萬元的預算。
Agent 3: 我推薦 AMD Ryzen 7 5800X。這款處理器擁有8核心16執行緒，適合多任務處理和高效能遊戲，性能優異。其基礎時脈為3.8GHz，最高可達4.7GHz，適合各種需求，並且在價格範圍內提供卓越的性價比。若你注重性能，這是一個很好的選擇。
